In [1]:
# To suppress Scikit-learn warnings
import warnings
warnings.filterwarnings('ignore')

# Loading and processing the data

In [2]:
import pandas as pd
data = pd.read_csv('./train.csv', parse_dates=['datetime'])
data.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,0.22,0.2727,0.80,0.0,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,0.22,0.2727,0.80,0.0,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,0.24,0.2879,0.75,0.0,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,0.24,0.2879,0.75,0.0,0,1,1


## Transforming raw features
In one way or another, virtually any machine learning method is based on recognizing and leveraging similarities between instances in the training data. For this to be successful, features of the instances should describe these similarities in a meaningful fashion. This does not only concern individual instances, but also relationships between them.

In this dataset, there are several examples of features that do not really satisfy this requirement, e.g., `weather`: is the distance between `weather=2` and `weather=4` really twice as big as between `weather=1` and `weather=2`? Similar reasoning applies to the raw `datetime` values.

First, let's break down `datetime` into higher-level components:

In [3]:
data['year']  = data['datetime'].dt.year
data['month'] = data['datetime'].dt.month
data['dow']   = data['datetime'].dt.dayofweek
data['hour']  = data['datetime'].dt.hour

data[['datetime', 'dow', 'year', 'month', 'hour']].sample(n=5)

,datetime,dow,year,month,hour
573,2011-02-07 01:00:00,0,2011,2,1
4519,2011-11-01 09:00:00,1,2011,11,9
8429,2012-07-12 14:00:00,3,2012,7,14
6047,2012-02-08 05:00:00,2,2012,2,5
6192,2012-02-14 06:00:00,1,2012,2,6


### One-hot encoding of categorical features
Now, such attributes as `year`, `month`, or `hour`, are meaningful on their own. On the contrary, just like `weather`, `dow` (day of week) is not: is Saturday equal to two Thursdays? 

Such issues are solved by *one-hot encoding*, where a so-called *dummy* binary variable is introduced per value of a multi-valued categorical variable; for example, 7 dummy features for `dow`. Thus, a learning algorithm can identify whether the day of the week matters, but will not try to compare *between* days of the week.

In [4]:
dow_df = pd.get_dummies(data['dow'], prefix='dow')
dow_df.sample(n=5)

,dow_0,dow_1,dow_2,dow_3,dow_4,dow_5,dow_6
3963,0,0,0,0,1,0,0
3490,1,0,0,0,0,0,0
8976,0,0,0,1,0,0,0
6983,1,0,0,0,0,0,0
8536,0,1,0,0,0,0,0


In [5]:
# Doing the same for `weather`
wthr_df = pd.get_dummies(data['weather'], prefix='wthr')
wthr_df.sample(n=5)

,wthr_1,wthr_2,wthr_3,wthr_4
10851,1,0,0,0
6747,0,1,0,0
10638,0,0,1,0
5178,1,0,0,0
10396,1,0,0,0


Function `prepare_data` puts these transformations together, so that they can be conveniently applied to the *test* data as well as the training data: obviously, all features accessible at training time must also be accessible at prediction time.

In [6]:
def prepare_data(filename):
    data = pd.read_csv(filename, parse_dates=['datetime'])
    data['year']  = data['datetime'].dt.year
    data['month'] = data['datetime'].dt.month
    data['dow']   = data['datetime'].dt.dayofweek
    data['hour']  = data['datetime'].dt.hour
    
    # `get_dummies` implements one-hot encoding
    dow_df = pd.get_dummies(data['dow'], prefix='dow')
    wthr_df = pd.get_dummies(data['weather'], prefix='wthr')
    
    # Attach generated features to the original data frame
    data = pd.concat([data, dow_df, wthr_df], axis=1)

    # Three lines below essentially
    features = ['year', 'month', 'hour', 'holiday', 'workingday',
                'temp', 'atemp', 'humidity', 'windspeed']
    features += ['dow_%d'%i for i in range(7)]
    features += ['wthr_%d'%i for i in range(1, 5)]
    X = data[features].values

    y = None
    y_registered = None
    y_casual = None
    if 'count' in data.columns:
        y = data['count'].values
    if 'registered' in data.columns:
        y_registered = data['registered'].values
        y_casual = data['casual'].values
    
    return X, y, y_registered, y_casual

X, y, y_registered, y_casual = prepare_data('./train.csv')
X_test, _, _, _ = prepare_data('./test.csv')

# Model selection and evaluation
Inevitably, coming up with a good prediction model requires several iterations of feature engineering, learner selection, parameter tuning, etc. The key challenge in this iterative process is to avoid overfitting, i.e. memorizing the training set as opposed to generalizing. To accomplish this, models must be tested on independent, unseen data.

## Separate validation set
The easiest way to ensure this is to set a fixed part of the training set aside as an independent validation set, which models are never trained on. *Scikit-learn* provides a function for that, not so aptly called `train_test_split`.

In [7]:
from sklearn.cross_validation import train_test_split
X_train, X_validation, y_train, y_validation = train_test_split(X, y)
print (X_train.shape, y_train.shape)
print (X_validation.shape, y_validation.shape)

(8164, 20) (8164,)
(2722, 20) (2722,)


/cw/dtaijupiter/NoCsBack/dtai/evgeniya/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## Cross-validation
Using a fixed validation has a number of downsides. Most importantly, only a part of the data is used for testing, which decreases reliability of score estimates and increases sensitivity to the split. *Cross-validation* improves on this by averaging the performance over a number of different splits, also known as *folds*, albeit at higher computational costs.

In [8]:
from sklearn.cross_validation import cross_val_score

## Training and evaluating a model
We will evaluate the models using *Root Mean Squared Logarithmic Error*. This measure is not built in *Scikit-learn*. Nevertheless, it provides facilities to define and use new scoring function.

In [10]:
from sklearn.metrics import make_scorer
import numpy as np

def rmsle_func(y_actual, y_predicted):
    sle = (np.log(y_predicted+1) - np.log(y_actual+1))**2
    return np.sqrt(np.mean(sle))

rmsle_loss = make_scorer(rmsle_func, greater_is_better=False)

# Ensembles
The goal of ensemble methods is to combine the predictions of several base estimators built with a given learning algorithm in order to improve generalizability / robustness over a single estimator. 

## Boosting
In boosting methods, several base estimators are built sequentially and then combined in order to produce a powerful ensemble.

*AdaBoost* trains base models (e.g. `DecisionTreeRegressor` below) in sequence while simultaneously re-weighing instances based on how good their targets are to predict already, whereas *Random Forests* do so by sampling a subset of instances and attributes for each model.

In [11]:
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import  DecisionTreeRegressor

tree_model = DecisionTreeRegressor(max_depth=5)
adaboost_model = AdaBoostRegressor(DecisionTreeRegressor(max_depth=15), n_estimators=1000)

In [12]:
from sklearn.ensemble import RandomForestRegressor

params = {'n_estimators': 1000, 'max_depth': 15, 'random_state': 0, 'min_samples_split' : 5, 'n_jobs': -1}
rf_model = RandomForestRegressor(**params)

## Bagging
Bootstrap aggregating, also called *bagging*, is a machine learning ensemble meta-algorithm that fits base models each on random subsets of the original dataset and then aggregates their individual predictions (either by voting or by averaging) to form a final prediction. 

Bagging reduces variance and helps to avoid overfitting. Although it is usually applied to decision tree methods, it can be used with any type of method.

In [13]:
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import  DecisionTreeRegressor

bagging_model = BaggingRegressor(DecisionTreeRegressor(max_depth=15), n_estimators=1000)

## Evaluating the models

Note that to use cross-validation to compare performance of different models correctly, the folds must be identical for all of them. In *Scikit-learn*, this can be ensured by using the same *fold generator*, e.g. `KFold`.

In [14]:
from sklearn.cross_validation import KFold
cv_generator = KFold(len(X), n_folds=3, shuffle=True)

def print_cv_score(model, name):
    scores = cross_val_score(model, X, y, scoring=rmsle_loss, cv=cv_generator)
    mean_score = -scores.mean()
    print('%25s: RMSLE = %.4f'%(name, mean_score))

# Compare performance of different models:   
print_cv_score(tree_model,     'DecisionTreeRegressor')
print_cv_score(adaboost_model, 'AdaBoostRegressor')
print_cv_score(rf_model,       'RandomForestRegressor')
print_cv_score(bagging_model,   'BaggingRegressor')

    DecisionTreeRegressor: RMSLE = 0.6101
        AdaBoostRegressor: RMSLE = 0.4221
    RandomForestRegressor: RMSLE = 0.3389
         BaggingRegressor: RMSLE = 0.3373


## Using two models 

Sometimes the data is composed of subsets that are generated by widely different mechanisms. In such cases, one can break the prediction problem into multiple problems, treat them independently, and combine their results at the end.  

In the previous sessions, we observed a difference in patterns of demands of *registered* and *casual* customers. We can build two models for predicting these two values independently, and combine the results to get the final prediction. 

In [15]:
scores = []
for train_indices, validation_indices in cv_generator:
    rf_model.fit(X[train_indices], y_casual[train_indices])
    y_c_pred = rf_model.predict(X[validation_indices])
    
    rf_model.fit(X[train_indices], y_registered[train_indices])
    y_r_pred = rf_model.predict(X[validation_indices])
    
    y_pred = y_c_pred + y_r_pred
    y_pred[y_pred < 0] = 0
    score = rmsle_func(y[validation_indices], y_pred)
    
    scores.append(score)
print (np.mean(scores))    

0.32833365705
